In [27]:
import os
from dotenv import load_dotenv
from langchain_cohere import CohereEmbeddings
from langchain_astradb import AstraDBVectorStore
import pandas as pd
load_dotenv()

True

In [28]:
embeddings = CohereEmbeddings(
		model="embed-english-light-v3.0",
		cohere_api_key=os.getenv("COHERE_API_KEY"),
	)

In [29]:
Vectorstore = AstraDBVectorStore(
    collection_name = "langchain_tutorial",
    embedding= embeddings,
    token= os.getenv("ASTRA_DB_APLICATION_TOKEN")
    # endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")
)

In [30]:
print("AstraDB Vector Store initialized successfully")

AstraDB Vector Store initialized successfully


In [31]:
book_dataset = pd.read_csv("books.csv")
print(f"Loaded {len(book_dataset)} book records from dataset")

Loaded 6810 book records from dataset


## Creating documents of dataset

In [32]:
from langchain_core.documents import Document

In [33]:
documents = []

for index, row in book_dataset.iterrows():
    if pd.isna(row['description'] or row['description'].strip() == ""):
        continue
    
    page_content = f"Title: {row['title']}\n Description: {row['description']}"
    
    metadata = {
        "title": row['title'],
        "author": row['authors'],
        "publication_year": row['published_year'],
        "categories": row['categories'],
        "average_rating": row['average_rating']
    }
    metadata = {k: v for k, v in metadata.items() if v is not None}
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)
    

print(f"Created {len(documents)} Document objects from book dataset")


Created 6548 Document objects from book dataset


In [37]:
documents[:10]

[Document(metadata={'title': 'Gilead', 'author': 'Marilynne Robinson', 'publication_year': 2004.0, 'categories': 'Fiction', 'average_rating': 3.85}, page_content='Title: Gilead\n Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in 

In [38]:
iserted_count = Vectorstore.add_documents(documents[:10])
print(f"Inserted {iserted_count} documents into AstraDB Vector Store")

Inserted ['b8d70ad4cf73433595837d3e9c4f13b2', 'cf4a917503604c8dbb138bc1e2023a4e', '794d961dc21c48dc97b6b3d5aec186a3', '24cf8177d18143bfaeb13de1d2ddaa7c', '599b23e7740440f9878a04f8aedcfba8', '3bc5dabe0855406a9897194cfb07c71a', '2ab22465574640c78bebdfc91865e821', '1c3809722f5047069df24a7c5601eca1', 'b106741a6ded411b8b9b46e33b9842af', '8b63eb80b1a64de4973908e6e5d80b0d'] documents into AstraDB Vector Store


In [44]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = Vectorstore.as_retriever(search_kwargs={"k": 3 }) 

prompt_template = "Answer the user quesry based on the following context:\n\n{context}, if you don't know the answer, say you don't know. User query: {query}"

prompt = ChatPromptTemplate.from_template(prompt_template)

In [45]:
llm = ChatOpenAI(
    model = "arcee-ai/trinity-large-preview:free",
    temperature = 0.4,
    base_url = "https://openrouter.ai/api/v1",
    default_headers = {
        "HTTP-Referer": "http://localhost",
        "X-Title": "LangChain Learning Project"
    }
)

In [46]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [47]:
query = "What are some good books to read if I like science fiction and fantasy genres?"
response = chain.invoke(query)
print("Response:", response)

Response: If you enjoy science fiction and fantasy, here are some highly recommended books and series:

1. **The Gap Series** by Stephen R. Donaldson - This includes "The Gap Into Madness" and other books in the series, which blend science fiction with complex characters and intricate plots.

2. **The Thomas Covenant Chronicles** by Stephen R. Donaldson - Another series by Donaldson, known for its dark fantasy elements and compelling anti-hero, Thomas Covenant.

3. **The One Tree** by Stephen R. Donaldson - Part of the second trilogy featuring Thomas Covenant, this book continues the dark fantasy saga.

4. **Empires of the Monsoon** by Richard Hall - While not strictly science fiction or fantasy, this historical book offers a fascinating look at the exploration and exploitation of the Indian Ocean, which can appeal to readers who enjoy world-building and epic narratives.

These books offer a mix of science fiction and fantasy elements, with rich storytelling and complex characters.
